In [51]:
// %help
// %ls

## variable zero value

[The_zero_value](https://go.dev/ref/spec#The_zero_value):
- booleans: `false`
- numeric types: `0`
- strings: `""`
- pointers, functions, interfaces, slices, channels, and maps: `nil`

关于 `nil`:
- nil is an untyped, predeclared identifier
- nil is a non-constant expression, assignable to or convertible(Type conversions `T(v)`) to any type whose zero value is nil


## Type conversions/assertions/switches

- [Type conversions](https://go.dev/ref/spec#Conversions): `T(x)`: 将一个具体类型转换为另一个具体类型, *statically checked*
- [Type assertions](https://go.dev/ref/spec#Type_assertions): `y, ok := x.(T)`: 将一个接口类型恢复成具体类型, x 必须是接口类型, x 实现 T 则 ok=true
  - [Type switches](https://go.dev/ref/spec#Type_switches): `switch x.(type) { /* cases */ }`

### 1. Type conversions

关于与 nil 的特殊用法示例(不建议):
- `[]int(nil)`: 一个 zerod slice
- `var _ io.Reader = (*myStruct)(nil)`: (编译期)保证 myStruct 实现 io.Reader

In [13]:
%main
var i float32 = 10

i32 := int32(i)
fmt.Printf("i: %v %T, i32: %v %T\n", i, i, i32, i32)

// type conversion 会进行静态检查，不通过则编译错误
//istr := string(i) // cannot convert i (variable of type float32) to type string

i: 10 float32, i32: 10 int32


### 2. Type assertions

In [55]:
%%
var i float32 = 10.1

// 必须是接口，所以 i.(string) 编译不通过
var iany any = i

// any(float32) to string, ok=false, float32 not implement string
_, okstr := iany.(string)
fmt.Printf("any(float32) to string. ok=%v\n", okstr)

// any(float32) to float64, ok=false, float32 not implement float64
_, okfloat64 := iany.(float64)
fmt.Printf("any(float32) to float64. ok=%v\n", okfloat64)

// 接口类型恢复成具体类型
_, okfloat32 := iany.(float32)
fmt.Printf("any(float32) back to float32. ok=%v\n", okfloat32)

any(float32) to string. ok=false
any(float32) to float64. ok=false
any(float32) back to float32. ok=true


## Type declarations

[Type declarations](https://go.dev/ref/spec#Type_declarations)
- Type Alias: 不会创新新类型，只是创建一个别名指向原类型，和原类型之间赋值不需要转换
  - Go 1.18 之后内置的 `type any = interface{}`
- Type definitions: 创建了一个新类型，和原类型之间赋值需要转换，不会继承原类型绑定的方法，但原类型是接口或者复合类型时特殊(原文: It does not inherit any methods bound to the given type, but the method set of an interface type or of elements of a composite type remains unchanged):
  - 原类型是接口则保有接口方法集合
  - 原类型是结构体内的元素则保有其方法


### 1. 区别1-和原类型之间赋值是否需要转换

In [77]:
%%

type myInt_alias = int
var numInt_alias myInt_alias = 1
// Type Alias 转原类型不需要转换
var numInt1 int = numInt_alias
fmt.Println(numInt_alias, numInt1)

type myInt_def int
var numInt_def myInt_def = 1
// var numInt int = numInt_def // 编译错误: cannot use numInt_def (variable of type myInt_def) as int value in variable declaration
// Type definitions 转原类型需要转换
var numInt2 int = int(numInt_def)
fmt.Println(numInt_def, numInt2)

1 1
1 1


### 2. 区别2-是否继承原类型方法/接口方法/复合类型字段

不会继承原类型绑定的方法:

In [82]:
type RedisLock struct {
    key string
}
func (m *RedisLock) Lock() { /* Lock implementation */ }
func (m *RedisLock) Unlock()  { /* Unlock implementation */ }

type RedisLock_def RedisLock

func main() {
    lock_org := RedisLock{key: "k-1"}
    lock_org.Lock()

    var lock_def RedisLock_def = RedisLock_def{key: "k-1"}
    // 不会继承原类型绑定的方法
    // lock_def.Lock() // 编译错误: lock_def.Lock undefined (type RedisLock_def has no field or method Lock)
    fmt.Printf("lock: %v\n", lock_def)
}


lock: {k-1}


结构体内的元素则保有其方法:

In [93]:
type RedisLock struct {
    key string
}
func (m *RedisLock) Lock() { /* Lock implementation */ }
func (m *RedisLock) Unlock()  { /* Unlock implementation */ }

type DistributedLock struct {
    RedisLock
}

func main() {
    lock_org := RedisLock{key: "k-1"}
    lock_org.Lock()

    var lock_def DistributedLock = DistributedLock{lock_org}
    // 结构体内的元素则保有其方法
    lock_def.Lock()
    fmt.Printf("lock: %v\n", lock_def)
}


lock: {{k-1}}


保有接口方法集合:

In [87]:
type Block interface {
	Size() int
	Encrypt(src []byte)
}

type Block_def Block

var block_def Block_def

func main() {
    // 保有接口方法集合
    // block_def.Size()
}

## todo types

[Types](https://go.dev/ref/spec#Types)